In [1]:
# Install required packages (run this cell in Google Colab)
!pip install mediapipe opencv-python-headless tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 43.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.6 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tens

In [4]:
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

# Initialize Mediapipe Face Mesh for video
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,   # For video, use dynamic mode
    max_num_faces=1,           # Process one face per frame
    refine_landmarks=True,     # Provides additional landmark details
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Define Mediapipe Face Mesh indices for the eyes (468-point model)
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]

# EAR threshold (tune this value as needed)
EAR_THRESHOLD = 0.2

def compute_ear(landmarks, eye_indices, image_width, image_height):
    """
    Compute Eye Aspect Ratio (EAR) for one eye.
    landmarks: list of face landmarks (normalized)
    eye_indices: list of 6 landmark indices for the eye.
    Returns: (ear, eye_coords) where ear is the computed value,
             and eye_coords is a list of pixel coordinates for the eye landmarks.
    """
    coords = []
    for idx in eye_indices:
        lm = landmarks[idx]
        x = int(lm.x * image_width)
        y = int(lm.y * image_height)
        coords.append((x, y))
    coords = np.array(coords)
    # Assign points: p1, p2, p3, p4, p5, p6 as per the order in eye_indices
    p1, p2, p3, p4, p5, p6 = coords
    vertical1 = np.linalg.norm(p2 - p6)
    vertical2 = np.linalg.norm(p3 - p5)
    horizontal = np.linalg.norm(p1 - p4)
    ear = (vertical1 + vertical2) / (2.0 * horizontal)
    return ear, coords

# Define input and output video paths
input_video = "/kaggle/input/variouslightenhacendvideos/bright low light eyes closed.mp4"    # Ensure your video file is uploaded to Colab
output_video = "output_videomedogk.mp4"  # Output video file name

# Open the input video
cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    print("Error: Could not open video file:", input_video)
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize VideoWriter for the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

pbar = tqdm(total=total_frames, desc="Processing frames")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame from BGR to RGB for Mediapipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            # Compute overall face bounding box from all landmarks
            xs = [lm.x for lm in landmarks]
            ys = [lm.y for lm in landmarks]
            min_x = int(min(xs) * width)
            max_x = int(max(xs) * width)
            min_y = int(min(ys) * height)
            max_y = int(max(ys) * height)
            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            cv2.putText(frame, "Face", (min_x, min_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            
            # Process left eye
            left_ear, left_eye_points = compute_ear(landmarks, LEFT_EYE_IDX, width, height)
            # Draw left eye landmarks and bounding box
            for (x, y) in left_eye_points:
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            left_eye_np = np.array(left_eye_points)
            lx, ly, lw, lh = cv2.boundingRect(left_eye_np)
            cv2.rectangle(frame, (lx, ly), (lx+lw, ly+lh), (0, 255, 0), 2)
            cv2.putText(frame, "Left Eye", (lx, ly - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
            # Process right eye
            right_ear, right_eye_points = compute_ear(landmarks, RIGHT_EYE_IDX, width, height)
            # Draw right eye landmarks and bounding box
            for (x, y) in right_eye_points:
                cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
            right_eye_np = np.array(right_eye_points)
            rx, ry, rw, rh = cv2.boundingRect(right_eye_np)
            cv2.rectangle(frame, (rx, ry), (rx+rw, ry+rh), (0, 0, 255), 2)
            cv2.putText(frame, "Right Eye", (rx, ry - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            
            # Determine overall eye state (using average EAR)
            overall_ear = (left_ear + right_ear) / 2.0
            if overall_ear < EAR_THRESHOLD:
                state = "Drowsy"
                state_color = (0, 0, 255)  # Red for drowsy
            else:
                state = "Awake"
                state_color = (0, 255, 0)  # Green for awake

            # Display the message on top left of the frame
            cv2.putText(frame, state, (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, state_color, 2)
    
    # Write the annotated frame to the output video
    out.write(frame)
    pbar.update(1)

pbar.close()
cap.release()
out.release()
face_mesh.close()
print("Processing complete. Output saved as:", output_video)

Processing frames: 100%|██████████| 136/136 [00:05<00:00, 26.10it/s]

Processing complete. Output saved as: output_videomedogk.mp4
